In [3]:
!pwd

/Users/alessandrorinaldi/code/Pr4t3/DriverDrowsinessDetector/notebooks


## Model Testing

In [1]:
import os
import tensorflow
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model


def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical' ):

    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=target_size)

In [3]:
BS= 32
TS=(24,24)
train_batch= generator('../raw_data/eyes/train_eyes',shuffle=True, batch_size=BS,target_size=TS)
valid_batch= generator('../raw_data/eyes/test_eyes',shuffle=True, batch_size=BS,target_size=TS)
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.
38 6


In [4]:
train_batch.classes[2000]

IndexError: index 2000 is out of bounds for axis 0 with size 1234

In [5]:
train_batch

In [ ]:
train_batch.

In [43]:
valid_batch[0][0][0].shape

(24, 24, 1)

In [30]:
img,labels= next(train_batch)
print(img.shape)

(32, 24, 24, 1)


In [9]:
# Model 
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#32 convolution filters used each of size 3x3
#again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),

#64 convolution filters used each of size 3x3
#choose the best features via pooling
    
#randomly turn neurons on and off to improve convergence
    Dropout(0.25),
#flatten since too many dimensions, we only want a classification output
    Flatten(),
#fully connected to get all relevant data
    Dense(128, activation='relu'),
#one more dropout for convergence' sake :) 
    Dropout(0.5),
#output a softmax to squash the matrix into output probabilities
    Dense(2, activation='softmax')
])

In [10]:
recall = tensorflow.keras.metrics.Recall(
    thresholds=None, top_k=None, class_id=None, name=None, dtype=None
)

In [11]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=[recall])

In [12]:
model.fit_generator(train_batch, validation_data=valid_batch,epochs=15,steps_per_epoch=SPE ,validation_steps=VS)

/Users/alessandrorinaldi/.pyenv/versions/3.8.12/envs/DriverDrowsinessDetector/lib/python3.8/site-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-03-02 16:55:20.060241: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
38/38 [==============================] - 14s 89ms/step - loss: 0.5876 - recall_1: 0.6211 - val_loss: 0.1899 - val_recall_1: 0.8108
Epoch 2/15
38/38 [==============================] - 2s 65ms/step - loss: 0.2179 - recall_1: 0.8337 - val_loss: 0.1663 - val_recall_1: 0.8676
Epoch 3/15
38/38 [==============================] - 2s 62ms/step - loss: 0.1704 - recall_1: 0.8764 - val_loss: 0.1402 - val_recall_1: 0.8941
Epoch 4/15
38/38 [==============================] - 2s 64ms/step - loss: 0.1198 - recall_1: 0.8993 - val_loss: 0.1026 - val_recall_1: 0.9102
Epoch 5/15
38/38 [==============================] - 2s 62ms/step - loss: 0.0883 - recall_1: 0.9137 - val_loss: 0.1142 - val_recall_1: 0.9207
Epoch 6/15
38/38 [==============================] - 2s 63ms/step - loss: 0.0933 - recall_1: 0.9232 - val_loss: 0.1103 - val_recall_1: 0.9281
Epoch 7/15
38/38 [==============================] - 2s 58ms/step - loss: 0.0534 - recall_1: 0.9301 - val_loss: 0.1441 - val_recall_1: 0.9346
Epoch 8/15
3

In [16]:
model.save('../models/model_eyes.h5', overwrite=True)

In [36]:
#model.fit(train_batch, y=None, validation_data=valid_batch,epochs=15,steps_per_epoch=SPE)

Epoch 1/15
77/77 [==============================] - 6s 79ms/step - loss: 0.4266 - recall_1: 0.7225
Epoch 2/15
77/77 [==============================] - 6s 77ms/step - loss: 0.3478 - recall_1: 0.7293
Epoch 3/15
77/77 [==============================] - 6s 75ms/step - loss: 0.3127 - recall_1: 0.7367
Epoch 4/15
77/77 [==============================] - 6s 72ms/step - loss: 0.2912 - recall_1: 0.7444
Epoch 5/15
77/77 [==============================] - 5s 71ms/step - loss: 0.2543 - recall_1: 0.7523
Epoch 6/15
77/77 [==============================] - 6s 72ms/step - loss: 0.2243 - recall_1: 0.7602
Epoch 7/15
77/77 [==============================] - 6s 72ms/step - loss: 0.2099 - recall_1: 0.7681
Epoch 8/15
57/77 [=====================>........] - ETA: 1s - loss: 0.1536 - recall_1: 0.7754


KeyboardInterrupt



/Users/alessandrorinaldi/.pyenv/versions/3.8.12/envs/DriverDrowsinessDetector/lib/python3.8/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


## Notes 

In [ ]:
y_pred = model.predict_classes(valid_batch[0][0])
y_pred


y_pred = np.argmax(model.predict(valid_batch[0][0]), axis=-1)
y_pred

In [ ]:

# Computing the probabilities of belonging to one of the 4 classes for the 10 000 images
predicted_probabilities = model.predict(X_test) # shape = 10_000 X 4

# Computing the predicted classes for all the images
predicted_classes = np.argmax(predicted_probabilities, axis = 1) # shape = 10_000 , 

# Confusion Matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, predicted_classes)
disp = ConfusionMatrixDisplay(confusion_matrix = cm,
                              display_labels = np.arange(0,4+1));

fig, ax = plt.subplots(figsize=(10,10));
disp.plot(cmap = "PuRd", ax = ax);